## Problem 1: Generating Random Boolean Functions

### Description
The idea for this problem is to randomly generate either a constant value (`True` or `False`) for all possible combinations of n binary inputs, or return True (which are randomly distributed) for exactly half of the combinations (and False for the other half).

For n inputs there will be `2ⁿ` outputs as there are `2ⁿ` combinations.

In [11]:
# Required imports
import random
import numpy as np
from itertools import product

In [12]:
def random_constant_balanced(num_inputs):
    table_size = 2 ** num_inputs # This will always be even, so we do not have to worry about uneven True and False counts.
    
    # Randomly choose between 'constant' and 'balanced'
    if random.choice(['constant', 'balanced']) == 'constant':
        return np.full(table_size, random.choice(['0', '1']))
    else:
        half = table_size // 2
        result = np.array(['0'] * half + ['1'] * half)
        np.random.shuffle(result)
        return result

In [13]:
def test_random_constant_balanced():
    for num_inputs in range(1, 21):  # Testing for 1 to 20 inputs
        for _ in range(10):  # Run multiple tests for each input size
            table = random_constant_balanced(num_inputs)
            assert len(table) == 2 ** num_inputs, f"Failed length test for {num_inputs} inputs"
            zeros = np.sum(table == '0')
            ones = np.sum(table == '1')
            assert zeros + ones == len(table), f"Failed value test for {num_inputs} inputs"
            assert zeros == ones or zeros == 0 or ones == 0, f"Failed balance/constant test for {num_inputs} inputs"
    print("All tests passed.")

In [14]:
test_random_constant_balanced()

All tests passed.


## Problem 2: Classical Testing for Function Type

### Description
This problem requires writing a function called `determine_constant_balanced`, which takes a parameter `f` (the function written in Problem 1). `f` is the truth table returned from the function.

`determine_constant_balanced` returns whether the function is `constant` or `balanced` based on the black box output.

### Solution Explanation

Since constant can either return True or False for every index in the truth table, we can take the initial value and save it in a variable `first_value`. We then loop through each index and if any index does not equal `first_value`, then the function must be `balanced`. If the first `len(f) // 2 + 1` indices of the truth table `f` all equal `first_value`, then the function is guaranteed to be constant, as more than half of the truth table is equal to `first_value`. Note that `len(f) // 2 + 1` can be expressed as `2ⁿ⁻¹ + 1` where `n` is the `num_inputs`. 

In [15]:
def determine_constant_balanced(f):
    """
    f is the truth table array from Problem 1.
    Returns 'constant' or 'balanced'.
    """

    # First value to compare to other indices.
    first_value = f[0]
    
    # Check only half the table + 1 since balanced means equal counts.
    for i in range(1, len(f) // 2 + 1):
        if f[i] != first_value:
            return 'balanced'  # Found a different value -> method must be balanced.
    
    return 'constant'  # All values were the same -> method must be constant.

### Testing